# Data cleaning of BeerAdvocate dataset

BeerAdvocate is a popular platform for beer enthusiasts to rate and review beers. The dataset provides information about beers, breweries, users and the ratings of the users for the beers on the platform.

On the website of BeerAdvocate, users can create an account. After creating an account with a username and password, they are asked to provide their location, but this is optional and not verified. 

Users can enter a beer into the database. Upon entering a beer, they have to select the location of the brewery where the beer is produced. They are then able to provide the beer name, the beer style and optionally an ABV (alcohol percentage).

After entering the beer into the database, every user can rate this beer. There are two types of ways a user can express their opinion on a beer: through ratings and reviews. A rating (in the current version of the platform, [the rating system has received a major update in 2014](https://www.beeradvocate.com/community/threads/beeradvocate-returns-to-its-roots-rating-system-revamped.238804/)) consists of Look, Smell, Taste, Feel and an Overall rating. Ratings range from 1-5 and can take quarter points as well (e.g. 3.75). Given that the dataset contains data that has been acquired before and after the major update the reviews are slightly different over time.

They can then optionally add a review as well. A review is a text description of what the user thinks about the beer. In some cases, as expressed in the article, the users must insert a text review if the reviews are outliers, to ensure that fraudulent reviews are reduced.

Our sources to better study this dataset are the following:
- [BeerAdvocate's forum](https://www.beeradvocate.com/community/threads/new-ba-score-beer-ranking-more-updates.537406/)
- [BeerAdvocate's website](https://www.beeradvocate.com)

## Preliminary work

### Install and import all the needed libraries

In [1]:
# Import all the necessary libraries
import polars as pl
import tqdm
import os

# Import the file in the utils folder
import sys
sys.path.append('../../utils')
from data_desc import remove_whitespaces, describe_numbers, describe

In [2]:
# Define the paths
SRC_DATA_PATH = '../../../data/BeerAdvocate'
DST_DATA_PATH = '../../../data/BeerAdvocate/processed'

# Create the DST_DATA_PATH if it does not exist
if not os.path.exists(DST_DATA_PATH):
    os.makedirs(DST_DATA_PATH)

## Data exploration

In [3]:
df_beers = remove_whitespaces(pl.read_csv(f"{SRC_DATA_PATH}/beers.csv"))
df_breweries = remove_whitespaces(pl.read_csv(f"{SRC_DATA_PATH}/breweries.csv"))
df_users = remove_whitespaces(pl.read_csv(f"{SRC_DATA_PATH}/users.csv"))
df_ratings = remove_whitespaces(pl.read_parquet(f"{SRC_DATA_PATH}/ratings.pq"))

##### Beers dataset
We'll start by looking at the beers dataset.

In [4]:
df_beers.sample(5)

beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
i64,str,i64,str,str,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64
133907,"""Eric's Peat-Smoked Beer""",26824,"""The Civil Life Brewing Company""","""Smoked Beer""",1,1,3.57,null,null,5.1,3.57,null,0,null
74650,"""Phros""",2299,"""Brouwerij Strubbe""","""Belgian Pale Ale""",1,1,4.0,null,null,7.0,4.0,null,0,null
44653,"""Bravo""",392,"""Weyerbacher Brewing Co.""","""American Pale Wheat Ale""",21,17,3.54,83.0,null,6.4,3.54381,-0.476442,17,3.642353
7821,"""Pumpviken Påsköl""",3476,"""Nynäshamns Ångbryggeri""","""English Strong Ale""",21,15,3.6,83.0,null,5.9,3.561905,null,0,null
200678,"""Pipkin""",38998,"""Folly Brewpub""","""Saison / Farmhouse Ale""",1,0,3.65,null,null,null,3.65,null,0,null


The dataset has the following structure

| Column Name | Description | 
|-------------|-------------|
| `beer_id` | Unique identifier for each beer |
| `beer_name` | Name of the beer |
| `brewery_id` | Unique identifier for the brewery that produced the beer |
| `brewery_name` | Name of the brewery that produced the beer |
| `style` | Style or category of the beer (e.g., IPA, Stout, Lager) |
| `nbr_ratings` | Number of ratings (text + non text) the beer has received |
| `nbr_reviews` | Number of written reviews (only text) the beer has received |
| `avg` | Average rating of the beer |
| `ba_score` | BeerAdvocate score for the beer (fraction of raters who gave the beer a 3.75 or higher) |
| `bros_score` | Score given by the Bros (Todd and Jason Alström, the BeerAdvocate founders) |
| `abv` | Alcohol By Volume percentage of the beer |
| `avg_computed` | Computed average rating (in some cases it differ from `avg` due to different calculation methods) |
| `zscore` | Standardized score indicating how many standard deviations the beer's rating is from the mean |
| `nbr_matched_valid_ratings` | Number of matched valid ratings |
| `avg_matched_valid_ratings` | Average of matched valid ratings |

Now that we have an idea of what the structure of our dataset looks like, let's see if our dataset is complete or whether it contains a lot of missing entries. 

In [5]:
describe(df_beers)

+---------------------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column                    | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|---------------------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| beer_id                   | Int64   |           280823 |             0 |    0.00 % |                280823 |          100.00 % |
| beer_name                 | String  |           280823 |             0 |    0.00 % |                236090 |           84.07 % |
| brewery_id                | Int64   |           280823 |             0 |    0.00 % |                 14325 |            5.10 % |
| brewery_name              | String  |           280823 |             0 |    0.00 % |                 14098 |            5.02 % |
| style                     | String  |           280823 |             0 |    0.00 

In [6]:
describe_numbers(df_beers, filters=["beer_id", "brewery_id"])

+---------------------------+-----------+----------+-----------+-----------+-----------+---------+---------+
| Column                    |      Mean |      Std |       25% |       50% |       75% |     Min |     Max |
|---------------------------+-----------+----------+-----------+-----------+-----------+---------+---------|
| nbr_ratings               |   29.8873 |   231.01 |         1 |         2 |         8 |       0 |   16509 |
| nbr_reviews               |   9.22142 |  68.8664 |         0 |         1 |         2 |       0 |    3899 |
| avg                       |   3.72103 | 0.476003 |       3.5 |      3.78 |      4.01 |       0 |       5 |
| ba_score                  |   84.6333 |  4.05272 |        83 |        85 |        86 |      46 |     100 |
| bros_score                |   84.8066 |  10.5077 |        81 |        87 |        91 |      31 |     100 |
| abv                       |   6.49137 |  2.05407 |         5 |         6 |       7.5 |    0.01 |    67.5 |
| avg_computed     

We can see that:
- The beer name and brewery information are present for every beer within the dataset
- Some data are missing for abv, ba_score, bros_score and the other aggregated scores
- The ba_score and the bros_score are in a 0 - 100 range
- The ba_score and bros_score have a very similar mean, but the bros_score has a far larger spread. This seems to signal that users of the website tend to give less extreme ratings that the bros (the owners of the website)
- The avg score is in a 0 - 5 range (meaning that the scores are in a 0 - 5 scale)
- The abv at first seems to have some outliers. The max in our table above shows we have values with ABV of 67.5 percent.

However, after careful analysis of the beers with such high ABV they actually exist and are not outliers. For example, the 'Brewdog Sink the Bismarck!' beer has 41% ABV and the 'Brewmeister Armageddon' has 65%. These are therefore perfectly fine entries and we will keep them in our dataframe.

Now we'll focus on dealing with the missing values. <br><br>
Regarding the ABV values, we will not be filling the NaN values here. This is due to the fact that it's not something that can be easily guessed and by approximating it with the mean or median values we would introduce a significant bias in the dataset.

In [7]:
# Compute the percentage of missing values for the reviews dataset
rev_nan_beers = df_beers.filter(pl.col("abv").is_null()).select(pl.col("nbr_ratings")).sum().item()
tot_rev = df_beers.select(pl.col("nbr_ratings")).sum().item()

# Compute the percentage of missing values for the reviews dataset
print(f"Percentage of missing values in the 'abv' column: {rev_nan_beers / tot_rev * 100:.2f}%")

Percentage of missing values in the 'abv' column: 2.04%


We see that by ignoring the beers with a nan abv we only ignore a small fraction of the reviews. This makes sense since it's unlikely that a beer with a high number of ratings would have a missing abv values.

Regarding the null values of the aggregated scores:
- In most cases these null values are associated with beers that doesn't have any review. It doesn't make sense to fill these values with the mean or median of the dataset since it would introduce a bias in the dataset.
- Most of the values will change after the processing and cleaning of the data.

For these two reasons we are not going to worry too much now about the missing values of the aggregated scores.

Since the beer are added by hand by people it can happen that a beer is created twice.

In [8]:
beers_duplicates = df_beers.filter(pl.struct(["beer_name", "brewery_id"]).is_duplicated())
beers_duplicates.head(5)

beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings
i64,str,i64,str,str,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64
207341,"""10|05 Coffee Porter (San Sebas…",33192,"""Brew By Numbers""","""English Porter""",1,0,4.05,null,null,6.5,4.05,null,0,null
255983,"""10|05 Coffee Porter (San Sebas…",33192,"""Brew By Numbers""","""English Porter""",1,0,4.13,null,null,6.5,4.13,null,0,null
27803,"""Nappa Scar""",12981,"""Yorkshire Dales Brewing Compan…","""Extra Special / Strong Bitter …",1,1,3.5,null,null,4.8,3.5,null,0,null
289247,"""Nappa Scar""",12981,"""Yorkshire Dales Brewing Compan…","""English Bitter""",1,0,3.5,null,null,4.0,3.5,null,0,null
259896,"""Sage Farm""",44830,"""Half Hours on Earth""","""Saison / Farmhouse Ale""",3,1,3.54,null,null,6.0,3.76,null,0,null


Since some of the values (e.g. the abv or the style) are not always consistent between duplicates, to reduce the risk of errors or bias we are going to just drop the duplicates.

In [9]:
dropped_beers_ids = beers_duplicates.select("beer_id").to_pandas().values.flatten()
df_beers = df_beers.filter(~pl.col("beer_id").is_in(dropped_beers_ids))

To better handle the data we are also going to split the nbr_ratings into two distinct columns:
- nbr_interactions: will be used to denote the number of interactions in total (number of text reviews + number of non text reviews)
- nbr_ratings: will be used to denote only the non text reviews

In [10]:
# Create a new column that is equals to nbr_ratings
df_beers = df_beers.with_columns(pl.col("nbr_ratings").alias("nbr_interactions"))
df_beers = df_beers.with_columns((pl.col("nbr_ratings") - pl.col("nbr_reviews")).alias("nbr_ratings"))
df_beers.sample(5)

beer_id,beer_name,brewery_id,brewery_name,style,nbr_ratings,nbr_reviews,avg,ba_score,bros_score,abv,avg_computed,zscore,nbr_matched_valid_ratings,avg_matched_valid_ratings,nbr_interactions
i64,str,i64,str,str,i64,i64,f64,f64,f64,f64,f64,f64,i64,f64,i64
142095,"""Oktoberhess""",33105,"""Mike Hess Brewing - North Park""","""Märzen / Oktoberfest""",2,0,3.81,null,null,5.9,3.81,null,0,null,2
25008,"""Bisschop Bier""",11891,"""Bierbrouwerij Mommeriete""","""Belgian Pale Ale""",0,1,3.13,null,null,6.8,3.13,null,0,null,1
62523,"""Cabernet Oak Aged Nihilist Imp…",2972,"""Mountain Sun Pub & Brewery""","""American Double / Imperial Sto…",0,1,3.87,null,null,10.0,3.87,null,0,null,1
292980,"""Slaapmutske Dubbel Glutenvrij""",4992,"""Brouwerij Slaapmutske""","""Dubbel""",1,0,3.78,null,null,7.4,3.78,-0.492944,0,null,1
256616,"""Guevara & Lynch Belgium Witbie…",40426,"""Desiderata Cerveza Artesanal |…","""Witbier""",0,1,2.75,null,null,null,2.75,null,0,null,1


##### Breweries dataset

Now we can take a look at our breweries dataframe. For each brewery we have a location, a name and the amount of beers they produce, along with a unique identifier. The brewery location can be very useful for our research questions. We will look into this more later on in this notebook.

In [11]:
df_breweries.sample(5)

id,location,name,nbr_beers
i64,str,str,i64
33560,"""United States, Iowa""","""SingleSpeed Brewing""",28
39073,"""Switzerland""","""Birra San Rocco""",6
44667,"""Spain""","""Besaro""",3
42654,"""United States, California""","""Skyland Ale Works""",9
13207,"""Slovak Republic""","""Pivovar Corgo&#328; (Heineken …",4


The dataset has the following structure
| Column Name | Description
|-------------|-------------|
| `id` | Unique identifier for each brewery |
| `location` | Geographic location of the brewery (country) |
| `name` | Name of the brewery |
| `nbr_beers` | Number of beers produced by the brewery |

In [12]:
describe(df_breweries)

+-----------+--------+------------------+---------------+-----------+-----------------------+-------------------+
| Column    | Type   |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|-----------+--------+------------------+---------------+-----------+-----------------------+-------------------|
| id        | Int64  |            16758 |             0 |    0.00 % |                 16758 |          100.00 % |
| location  | String |            16758 |             0 |    0.00 % |                   297 |            1.77 % |
| name      | String |            16758 |             0 |    0.00 % |                 16237 |           96.89 % |
| nbr_beers | Int64  |            16758 |             0 |    0.00 % |                   273 |            1.63 % |
+-----------+--------+------------------+---------------+-----------+-----------------------+-------------------+


In [13]:
describe_numbers(df_breweries, filters=["id"])

+-----------+---------+---------+-------+-------+-------+-------+-------+
| Column    |    Mean |     Std |   25% |   50% |   75% |   Min |   Max |
|-----------+---------+---------+-------+-------+-------+-------+-------|
| nbr_beers | 21.0563 | 69.4178 |     2 |     6 |    18 |     0 |  1196 |
+-----------+---------+---------+-------+-------+-------+-------+-------+


For our breweries, we have no missing values. <br><br>
From the analysis we can see that some of our breweries have a non-unique name. We need to verify if multiple breweries with the same name exists in the same country.

In [14]:
breweries_duplicates = df_breweries.filter(pl.struct(["name", "location"]).is_duplicated())
breweries_duplicates

id,location,name,nbr_beers
i64,str,str,i64
34598,"""Wales""","""Rhymney Brewery""",2
12936,"""Wales""","""Rhymney Brewery""",13
11410,"""England""","""Bartrams Brewery""",11
7095,"""England""","""Bartrams Brewery""",0
31304,"""England""","""Dorset Piddle Brewery""",0
…,…,…,…
16078,"""United States, Ohio""","""The Brew Keeper""",27
1127,"""United States, Georgia""","""Buckhead Brewery and Grill""",26
16099,"""United States, Florida""","""Spanish Springs Brewing Compan…",0


We see that there are some breweries that have the same name and are located in the same country. This introduces a possible source of error:
- Either multiple users have inserted the same brewery into the database multiple times
- Multiple breweries in one country have the same name

Both assumption are reasonable but we can't be sure which one is the correct one. To avoid introducing errors in our dataset we are going to drop the duplicates.

In [15]:
dropped_breweries_ids = breweries_duplicates.select("id")
df_breweries = df_breweries.filter(~pl.col("id").is_in(dropped_breweries_ids))

Since we modified both the breweries and the beers dataset let's recompute the shared values. At the same time we are going to drop the breweries that doesn't have any beer associated with them.

In [16]:
# Drop all the beers whose brewery has been dropped
df_beers = df_beers.filter(pl.col("brewery_id").is_in(df_breweries.select("id")))

# Recompute the number of beers per each brewery
aggregated_value = df_beers.group_by("brewery_id").agg(pl.col("beer_id").count().alias("beers_count")).rename({"brewery_id": "id"}).cast(pl.Int64)

# Add the new column to the breweries dataset
df_breweries = df_breweries.join(aggregated_value, on="id", how="inner")
df_breweries = df_breweries.drop("nbr_beers")

df_breweries

id,location,name,beers_count
i64,str,str,i64
39912,"""Kyrgyzstan""","""Abdysh-Ata (Абдыш Ата)""",5
39913,"""Kyrgyzstan""","""Arpa (АРПА)""",1
39914,"""Kyrgyzstan""","""Bear Beer""",4
39915,"""Kyrgyzstan""","""Blonder Pub""",4
39916,"""Kyrgyzstan""","""Kellers Bier""",2
…,…,…,…
10673,"""United States, Indiana""","""Warbird Brewing Company""",6
16233,"""United States, California""","""Farmhouse Brewing Co.""",4
1613,"""United States, Florida""","""Panama City Brewery & Cafe""",5


While we still have some beers with the same name we are sure that these beers are produced by different breweries. <br><br>
We are going to recompute the number of ratings, interactions and reviews for each beer later in this notebook.

##### Users dataset

For each of our users, we have their number of ratings/reviews, their ID, their name, the timestamp of when they joined and their location. The location is interesting for us, along with the number of reviews of each user.

In [17]:
df_users.sample(5)

nbr_ratings,nbr_reviews,user_id,user_name,joined,location
i64,i64,str,str,i64,str
4,0,"""feartheelf.655771""","""FearTheElf""",1327057200,"""United States, Ohio"""
26,23,"""abeerlovr.611411""","""abeerlovr""",1311328800,"""United States, Florida"""
12,1,"""isawpalmetto.253520""","""isawpalmetto""",1222596000,"""United States, Illinois"""
105,3,"""robam.873664""","""RobAM""",1412503200,null
10,0,"""ipedal88.903312""","""ipedal88""",1417777200,"""United States, California"""


This dataset has the following structure
| Column Name | Description 
|-------------|-------------|
| `user_id` | Unique identifier for each user |
| `user_name` | Username of the reviewer |
| `joined` | Date when the user joined BeerAdvocate |
| `location` | Geographic location of the user |
| `nbr_ratings` | Number of ratings submitted by the user |
| `nbr_reviews` | Number of written reviews submitted by the user |

In [18]:
describe(df_users)

+-------------+--------+------------------+---------------+-----------+-----------------------+-------------------+
| Column      | Type   |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|-------------+--------+------------------+---------------+-----------+-----------------------+-------------------|
| nbr_ratings | Int64  |           153704 |             0 |    0.00 % |                  2053 |            1.34 % |
| nbr_reviews | Int64  |           153704 |             0 |    0.00 % |                  1265 |            0.82 % |
| user_id     | String |           153704 |             0 |    0.00 % |                153704 |          100.00 % |
| user_name   | String |           153704 |             0 |    0.00 % |                153704 |          100.00 % |
| joined      | Int64  |           151052 |          2652 |    1.73 % |                  5525 |            3.66 % |
| location    | String |           122425 |         31279 |   20.35 % | 

In [19]:
describe_numbers(df_users)

+-------------+-------------+-------------+-------------+-------------+-------------+-----------+------------+
| Column      |        Mean |         Std |         25% |         50% |         75% |       Min |        Max |
|-------------+-------------+-------------+-------------+-------------+-------------+-----------+------------|
| nbr_ratings |     54.6052 |     252.389 |           1 |           3 |          16 |         1 |      12046 |
| nbr_reviews |     16.8479 |     139.847 |           0 |           0 |           2 |         0 |       8970 |
| joined      | 1.35724e+09 | 9.19513e+07 | 1.30312e+09 | 1.39194e+09 | 1.41769e+09 | 840794400 | 1501495200 |
+-------------+-------------+-------------+-------------+-------------+-------------+-----------+------------+


We are going to change the structure in nbr_interactions, nbr_ratings, nbr_reviews as done with the beers dataset and we are also going to cast the joined column to a datetime object.

In [20]:
df_users = df_users.with_columns(pl.col("nbr_ratings").alias("nbr_interactions"))
df_users = df_users.with_columns((pl.col("nbr_ratings") - pl.col("nbr_reviews")).alias("nbr_ratings"))
df_users = df_users.with_columns((pl.col("joined").cast(pl.Int64) * 1000).cast(pl.Datetime("ms")))
df_users.sample(5)

nbr_ratings,nbr_reviews,user_id,user_name,joined,location,nbr_interactions
i64,i64,str,str,datetime[ms],str,i64
0,4,"""bpwatl.254887""","""bpwatl""",2008-10-04 10:00:00,"""United States, Georgia""",4
14,0,"""francischarest.807999""","""francischarest""",2014-06-17 10:00:00,null,14
0,7,"""seebs847.235461""","""Seebs847""",2008-07-17 10:00:00,"""United States, Illinois""",7
1,0,"""irishfan.856052""","""Irishfan""",2014-08-31 10:00:00,null,1
1,0,"""ceriaz.675909""","""Ceriaz""",2012-05-09 10:00:00,null,1


We see that lots of users doesn't have a location associated with them and this is reasonable since the location is an optional field. Strangely enough some users doesn't have neither the timestamp of when they joined the platform. We are going to verify how many reviews and ratings are of these users to ensure that we still have enough temporal and spatial data to work with.

In [21]:
# Compute the percentage of missing values for the reviews dataset
tot_number_users = df_users.select("nbr_interactions").sum().item()
tot_users_no_location_nbr_interactions = df_users.filter(pl.col("location").is_null()).select("nbr_interactions").sum().item()
tot_users_no_timestamp_nbr_interactions = df_users.filter(pl.col("joined").is_null()).select("nbr_interactions").sum().item()
tot_users_either_of_two = df_users.filter(pl.col("location").is_null() | pl.col("joined").is_null()).select("nbr_interactions").sum().item()

# Compute the percentage of missing values for the reviews dataset
print(f"Percentage of interactions (reviews or ratings) with missing user location: {tot_users_no_location_nbr_interactions / tot_number_users * 100:.2f} %")
print(f"Percentage of interactions (reviews or ratings) with missing user timestamp: {tot_users_no_timestamp_nbr_interactions / tot_number_users * 100:.2f} %")
print(f"Percentage of interactions (reviews or ratings) with missing user location or timestamp: {tot_users_either_of_two / tot_number_users * 100:.2f} %")

Percentage of interactions (reviews or ratings) with missing user location: 5.96 %
Percentage of interactions (reviews or ratings) with missing user timestamp: 4.15 %
Percentage of interactions (reviews or ratings) with missing user location or timestamp: 5.96 %


We see that the number of reviews and ratings of people that doesn't have a location or a join date is very low. Additionally it's interesting to see that almost all of the users that doesn't have a location associated with them doesn't have a join date as well. We still have enough data for our purposes.

We are going to recompute the number of ratings, interactions and reviews for each beer later on when we will work with the ratings dataset.

##### Ratings

In [22]:
df_ratings.sample(5)

user_id,rating,review,abv,brewery_name,user_name,beer_id,appearance,palate,text,aroma,overall,taste,style,beer_name,brewery_id,date
str,f64,bool,f64,str,str,i64,f64,f64,str,f64,f64,f64,str,str,i64,datetime[μs]
"""shovelbum.38151""",2.16,true,5.0,"""Molson Coors Canada""","""Shovelbum""",1312,3.0,3.0,"""White foamy head over a clear …",2.0,2.0,2.0,"""American Adjunct Lager""","""Molson Canadian Lager""",433,2005-11-25 12:00:00
"""brocktune.762032""",3.75,false,11.1,"""Parkway Brewing Co.""","""brocktune""",99003,null,null,null,null,null,null,"""Belgian Strong Dark Ale""","""Magella Belgian Dark Abbey Ale""",30815,2014-11-26 12:00:00
"""hozersr.799427""",3.83,false,10.0,"""Oskar Blues Brewery & Tasty We…","""hozersr""",55412,4.0,3.75,null,4.0,3.75,3.75,"""American Double / Imperial IPA""","""GUBNA Imperial IPA""",2681,2015-10-20 12:00:00
"""jzlyo.663004""",4.0,false,6.4,"""Summit Brewing Company""","""jzlyo""",81072,null,null,null,null,null,null,"""American IPA""","""Summit Sága IPA""",467,2014-08-09 12:00:00
"""ihazabucket7.992436""",4.42,false,5.0,"""Mother Earth Brew Co.""","""ihazabucket7""",87942,5.0,4.5,"""Love, love, LOVE this beer. Va…",4.25,4.25,4.5,"""Cream Ale""","""Cali Creamin' Vanilla Cream Al…",22610,2015-06-28 12:00:00


The dataset has the following structure

In [23]:
describe(df_ratings)

+--------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------+
| Column       | Type                                     |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|--------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------|
| user_id      | String                                   |          8393032 |             0 |    0.00 % |                153704 |            1.83 % |
| rating       | Float64                                  |          8393032 |             0 |    0.00 % |                   401 |            0.00 % |
| review       | Boolean                                  |          8393032 |             0 |    0.00 % |                     2 |            0.00 % |
| abv          | Float64                                  |          8221727 |        171305 |

The dataset has the following structure
| Column Name | Description 
|-------------|-------------|
| `user_id` | Unique identifier for each user |
| `rating` | Global rating of the beer from the user |
| `review` | Flag to tell if the rating has text or not |
| `abv` | Alcohol By Volume percentage of the beer |
| `brewery_name` | Name of the brewery that produced the beer |
| `user_name` | Username of the reviewer |
| `beer_id` | Unique identifier for each beer |
| `appearance` | Rating of the appearance of the beer |
| `palate` | Rating of the palate of the beer |
| `text` | Text of the review |
| `aroma` | Rating of the aroma of the beer |
| `overall` | Overall rating of the beer |
| `taste` | Rating of the taste of the beer |
| `style` | Style or category of the beer (e.g., IPA, Stout, Lager) |
| `beer_name` | Name of the beer |
| `brewery_id` | Unique identifier for the brewery that produced the beer |
| `date` | Date when the review was submitted |

Since this dataset includes data from the breweries dataset, users dataset and beers dataset but we have modified them let's clean the reviews whose user_id, beer_id or brewery_id is not present in the respective datasets.

In [24]:
# Get the beers and breweries ids
beers_ids = df_beers.select("beer_id")
brewery_ids = df_breweries.select("id")

# Filter the ratings dataset
df_ratings = df_ratings.filter(pl.col("beer_id").is_in(beers_ids)).filter(pl.col("brewery_id").is_in(brewery_ids))

We are going to check if some of the reviews have a text but have the review flag as false

In [25]:
df_ratings_review_false_but_text = df_ratings.filter(~pl.col("review") & pl.col("text").is_not_null())
df_ratings_review_false_but_text.sample(5)

user_id,rating,review,abv,brewery_name,user_name,beer_id,appearance,palate,text,aroma,overall,taste,style,beer_name,brewery_id,date
str,f64,bool,f64,str,str,i64,f64,f64,str,f64,f64,f64,str,str,i64,datetime[μs]
"""vinnygny.990645""",4.88,false,6.9,"""Stone Brewing""","""Vinnygny""",88,4.75,5.0,"""Great IPA. I prefer Ruination …",4.75,4.75,5.0,"""American IPA""","""Stone IPA (India Pale Ale)""",147,2015-06-05 12:00:00
"""lightningholt.1173667""",4.49,false,11.0,"""Boiler Brewing Co.""","""Lightningholt""",229647,4.75,4.25,"""Love this beer. Rich, smooth, …",4.5,4.5,4.5,"""Russian Imperial Stout""","""Alcohall & Oates""",44994,2017-06-22 12:00:00
"""lettsgo3.807337""",1.0,false,4.2,"""Anheuser-Busch""","""Lettsgo3""",41821,1.0,1.0,"""The only thing is that it was …",1.0,1.0,1.0,"""Light Lager""","""Bud Light Lime""",29,2015-06-18 12:00:00
"""redneck45x.967128""",4.71,false,9.0,"""Dogfish Head Brewings & Eats""","""Redneck45x""",1587,1.75,4.0,"""Very good beer, sweet aroma an…",5.0,5.0,5.0,"""Herbed / Spiced Beer""","""Midas Touch""",64,2015-03-31 12:00:00
"""facetiousstudent.984653""",3.99,false,5.0,"""Anderson Valley Brewing Compan…","""Facetiousstudent""",127026,3.75,4.0,"""Great tasting beer. Tastes lik…",4.0,4.0,4.0,"""Fruit / Vegetable Beer""","""Bahl Hornin’ Summer Solstice""",193,2015-05-09 12:00:00


In [26]:
print("The percentage of reviews that are False but have a text is: {:.2f} %".format(df_ratings_review_false_but_text.shape[0] / df_ratings.shape[0] * 100))

The percentage of reviews that are False but have a text is: 1.50 %


While at first it seems like an error it's most likely the data where the review has been flagged as 'ambiguous' in the [paper](https://arxiv.org/pdf/1210.3926) so the text should be ignored. Since no strange results seems to be present we are not going to drop these results and we are just going to ignore the text for these reviews.

## Data cleaning and saving
Now that we have done some preliminary analysis on the data we will modify some of the datasets to make them more suitable for our analysis. We will also save the datasets in parquet format with the correct data types for faster loading in the future.
##### Users dataset
The dataset is already clean and processed, we just need to save it in parquet format

In [27]:
# Drop the nbr_ratings, nbr_reviews and nbr_interactions columns
df_users = df_users.drop(["nbr_ratings", "nbr_reviews", "nbr_interactions"])
    
# Select the ratings and the reviews
ratings = df_ratings.filter(pl.col("review"))
reviews = df_ratings.filter(~pl.col("review"))

# Get the user ids
number_of_ratings = ratings.group_by("user_id").agg(pl.col("rating").count().alias("nbr_ratings").cast(pl.Int64)).select("user_id", "nbr_ratings")
number_of_reviews = reviews.group_by("user_id").agg(pl.col("rating").count().alias("nbr_reviews").cast(pl.Int64)).select("user_id", "nbr_reviews")
number_of_interactions = df_ratings.group_by("user_id").agg(pl.col("rating").count().alias("nbr_interactions").cast(pl.Int64)).select("user_id", "nbr_interactions")

# Update the users dataset
df_users = df_users.join(number_of_ratings, on="user_id", how="left")
df_users = df_users.join(number_of_reviews, on="user_id", how="left")
df_users = df_users.join(number_of_interactions, on="user_id", how="left")

# Save the processed datasets
df_users.write_parquet(f"{DST_DATA_PATH}/users.pq")

In [28]:
df_users.sample(5)

user_id,user_name,joined,location,nbr_ratings,nbr_reviews,nbr_interactions
str,str,datetime[ms],str,i64,i64,i64
"""kinghefey.506376""","""KingHefey""",2010-09-23 10:00:00,"""Canada""",5,null,5
"""drew2430.1145887""","""Drew2430""",2016-05-18 10:00:00,"""United States, Texas""",null,1,1
"""saonbikes.902071""","""SAonbikes""",2014-12-02 11:00:00,null,null,1,1
"""jdagz84.951619""","""JDAGZ84""",2015-02-28 11:00:00,"""United States, New Jersey""",null,75,75
"""jlblakley.782364""","""jlblakley""",2014-02-15 11:00:00,null,null,5,5


In [29]:
describe(df_users)

+------------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------+
| Column           | Type                                     |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|------------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------|
| user_id          | String                                   |           153704 |             0 |    0.00 % |                153704 |          100.00 % |
| user_name        | String                                   |           153704 |             0 |    0.00 % |                153704 |          100.00 % |
| joined           | Datetime(time_unit='ms', time_zone=None) |           151052 |          2652 |    1.73 % |                  5525 |            3.66 % |
| location         | String                                   |       

In [30]:
describe_numbers(df_users)

+------------------+---------+---------+-------+-------+-------+-------+-------+
| Column           |    Mean |     Std |   25% |   50% |   75% |   Min |   Max |
|------------------+---------+---------+-------+-------+-------+-------+-------|
| nbr_ratings      | 41.0704 | 213.438 |     1 |     2 |    10 |     1 |  8874 |
| nbr_reviews      | 47.9458 | 213.759 |     1 |     3 |    15 |     1 | 10091 |
| nbr_interactions | 53.8145 |  247.61 |     1 |     3 |    16 |     1 | 11591 |
+------------------+---------+---------+-------+-------+-------+-------+-------+


##### Breweries dataset
The dataset is already clean and processed, we just need to save it in parquet format

In [31]:
df_breweries.write_parquet(f"{DST_DATA_PATH}/breweries.pq")

In [32]:
df_breweries

id,location,name,beers_count
i64,str,str,i64
39912,"""Kyrgyzstan""","""Abdysh-Ata (Абдыш Ата)""",5
39913,"""Kyrgyzstan""","""Arpa (АРПА)""",1
39914,"""Kyrgyzstan""","""Bear Beer""",4
39915,"""Kyrgyzstan""","""Blonder Pub""",4
39916,"""Kyrgyzstan""","""Kellers Bier""",2
…,…,…,…
10673,"""United States, Indiana""","""Warbird Brewing Company""",6
16233,"""United States, California""","""Farmhouse Brewing Co.""",4
1613,"""United States, Florida""","""Panama City Brewery & Cafe""",5


In [33]:
describe(df_breweries)

+-------------+--------+------------------+---------------+-----------+-----------------------+-------------------+
| Column      | Type   |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|-------------+--------+------------------+---------------+-----------+-----------------------+-------------------|
| id          | Int64  |            14028 |             0 |    0.00 % |                 14028 |          100.00 % |
| location    | String |            14028 |             0 |    0.00 % |                   277 |            1.97 % |
| name        | String |            14028 |             0 |    0.00 % |                 13966 |           99.56 % |
| beers_count | Int64  |            14028 |             0 |    0.00 % |                   270 |            1.92 % |
+-------------+--------+------------------+---------------+-----------+-----------------------+-------------------+


In [34]:
describe_numbers(df_breweries, filters=["id"])

+-------------+---------+---------+-------+-------+-------+-------+-------+
| Column      |    Mean |     Std |   25% |   50% |   75% |   Min |   Max |
|-------------+---------+---------+-------+-------+-------+-------+-------|
| beers_count | 19.5155 | 40.0882 |     3 |     8 |    20 |     1 |  1196 |
+-------------+---------+---------+-------+-------+-------+-------+-------+


##### Ratings dataset
The dataset is already cleaned and process, we just need to save it in parquet format. We will both save a full version of the dataset and a version without the text of the reviews.

In [35]:
df_ratings.write_parquet(f"{DST_DATA_PATH}/ratings.pq")
df_ratings_no_text = df_ratings.drop("text")
df_ratings_no_text.write_parquet(f"{DST_DATA_PATH}/ratings_no_text.pq")

In [36]:
df_ratings_no_text.sample(5)

user_id,rating,review,abv,brewery_name,user_name,beer_id,appearance,palate,aroma,overall,taste,style,beer_name,brewery_id,date
str,f64,bool,f64,str,str,i64,f64,f64,f64,f64,f64,str,str,i64,datetime[μs]
"""jsilva.391708""",4.25,false,7.0,"""The Bruery""","""jsilva""",70286,null,null,null,null,null,"""American Wild Ale""","""Tart Of Darkness""",16866,2013-02-24 12:00:00
"""baklo.698188""",3.75,false,6.5,"""New Belgium Brewing""","""baklo""",1910,null,null,null,null,null,"""Fruit / Vegetable Beer""","""Frambozen Raspberry Brown Ale""",192,2012-11-26 12:00:00
"""madelf66.855500""",3.91,false,6.0,"""Heavy Seas Beer""","""madelf66""",272955,3.75,3.75,4.0,3.75,4.0,"""American IPA""","""Single Cannon""",898,2017-07-26 12:00:00
"""teromous.440420""",3.85,true,3.8,"""Ballast Point Brewing Company""","""teromous""",30944,3.5,4.0,3.5,4.0,4.0,"""American Pale Ale (APA)""","""Even Keel Session IPA""",199,2016-12-04 12:00:00
"""natemac13.591479""",3.25,false,4.0,"""Brasserie Fantôme""","""NateMac13""",32708,null,null,null,null,null,"""Saison / Farmhouse Ale""","""Fantôme BBB Dark White""",738,2013-05-06 12:00:00


In [37]:
describe(df_ratings_no_text)

+--------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------+
| Column       | Type                                     |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|--------------+------------------------------------------+------------------+---------------+-----------+-----------------------+-------------------|
| user_id      | String                                   |          8229373 |             0 |    0.00 % |                152921 |            1.86 % |
| rating       | Float64                                  |          8229373 |             0 |    0.00 % |                   401 |            0.00 % |
| review       | Boolean                                  |          8229373 |             0 |    0.00 % |                     2 |            0.00 % |
| abv          | Float64                                  |          8064274 |        165099 |

In [38]:
describe_numbers(df_ratings_no_text, filters=["beer_id", "brewery_id", "user_id"])

+------------+---------+----------+-------+-------+-------+-------+-------+
| Column     |    Mean |      Std |   25% |   50% |   75% |   Min |   Max |
|------------+---------+----------+-------+-------+-------+-------+-------|
| rating     | 3.88197 |  0.62109 |  3.54 |     4 |  4.25 |     1 |     5 |
| abv        |   7.335 |  2.46824 |   5.5 |   6.9 |   8.8 |  0.01 |  67.5 |
| appearance | 3.93564 | 0.558574 |  3.75 |     4 |  4.25 |     1 |     5 |
| palate     | 3.86437 | 0.608519 |   3.5 |     4 |  4.25 |     1 |     5 |
| aroma      | 3.86777 |  0.62057 |   3.5 |     4 |  4.25 |     1 |     5 |
| overall    | 3.89903 | 0.616302 |   3.5 |     4 |  4.25 |     1 |     5 |
| taste      | 3.90194 | 0.643627 |   3.5 |     4 |  4.25 |     1 |     5 |
+------------+---------+----------+-------+-------+-------+-------+-------+


##### Beers dataset
While for the previous datasets not much work was needed here multiple steps are needed. In particular:
- We will change some of the columns of the dataset. In particular we are going to create a dataset that is similar to BeerAdvocate dataset to simplify the comparison between the two datasets.
- Since we have removed some reviews and changed other datasets we are going to recompute the values in the beer dataset

Additional data regarding the matching of the two datasets will be added in another notebook.

In [39]:
rows = []
for row in tqdm.tqdm(df_beers.rows(named=True)):
    # Create the new dictionary
    new_row = {}

    # Compute general information
    new_row["beer_id"] = row["beer_id"]
    new_row["beer_name"] = row["beer_name"]
    new_row["brewery_id"] = row["brewery_id"]
    new_row["brewery_name"] = row["brewery_name"]
    new_row["style"] = row["style"]
    new_row["abv"] = row["abv"]

    # Compute aggregated informations
    reviews_elements = df_ratings_no_text.filter(pl.col("beer_id") == row["beer_id"])
    new_row["rating_score_avg"] = reviews_elements["rating"].mean()
    new_row["rating_score_std"] = reviews_elements["rating"].std()
    new_row["rating_score_median"] = reviews_elements["rating"].median()

    # Compute the number of interactions, nbr_ratings and nbr_reviews (here they are the same)
    reviews_text = reviews_elements.filter(pl.col("review"))
    ratings = reviews_elements.filter(~pl.col("review"))
    new_row["nbr_interactions"] = reviews_elements.shape[0]
    new_row["nbr_ratings"] = ratings.shape[0]
    new_row["nbr_reviews"] = reviews_text.shape[0]

    # Append the new row to the list of rows
    rows.append(new_row)

# Transform the data into a Polars DataFrame
df_beers = pl.DataFrame(rows)

# Save the data
df_beers.write_parquet(f"{DST_DATA_PATH}/beers.pq")

100%|██████████| 273764/273764 [09:53<00:00, 461.04it/s]


In [40]:
df_beers.sample(5)

beer_id,beer_name,brewery_id,brewery_name,style,abv,rating_score_avg,rating_score_std,rating_score_median,nbr_interactions,nbr_ratings,nbr_reviews
i64,str,i64,str,str,f64,f64,f64,f64,i64,i64,i64
49528,"""Brooklyn American Ale""",45,"""Brooklyn Brewery""","""American Pale Ale (APA)""",4.5,3.373109,0.44547,3.47,119,95,24
87450,"""Phantom Punch Winter Stout""",23922,"""Baxter Brewing Co.""","""Foreign / Export Stout""",6.8,3.750449,0.484156,3.75,312,256,56
199682,"""Leinenkugel's Spiced Pear Shan…",710,"""Jacob Leinenkugel Brewing Comp…","""Fruit / Vegetable Beer""",4.2,3.2625,0.676671,3.395,32,19,13
95690,"""Mandarina Single Hop IPA""",13307,"""Mikkeller ApS""","""American IPA""",6.5,3.807143,0.164186,3.84,7,3,4
152275,"""Category 4 IPA""",25168,"""Holy City Brewing""","""American IPA""",8.0,null,null,null,0,0,0


In [41]:
describe(df_beers)

+---------------------+---------+------------------+---------------+-----------+-----------------------+-------------------+
| Column              | Type    |   Not null count |   Nulls count |   Nulls % |   Unique values count |   Unique values % |
|---------------------+---------+------------------+---------------+-----------+-----------------------+-------------------|
| beer_id             | Int64   |           273764 |             0 |    0.00 % |                273764 |          100.00 % |
| beer_name           | String  |           273764 |             0 |    0.00 % |                231522 |           84.57 % |
| brewery_id          | Int64   |           273764 |             0 |    0.00 % |                 14028 |            5.12 % |
| brewery_name        | String  |           273764 |             0 |    0.00 % |                 13965 |            5.10 % |
| style               | String  |           273764 |             0 |    0.00 % |                   104 |            0.04 % |


In [42]:
describe_numbers(df_beers, filters=["beer_id", "brewery_id"])

+---------------------+----------+----------+----------+----------+----------+-------+---------+
| Column              |     Mean |      Std |      25% |      50% |      75% |   Min |     Max |
|---------------------+----------+----------+----------+----------+----------+-------+---------|
| abv                 |  6.49229 |  2.05808 |        5 |        6 |      7.5 |  0.01 |    67.5 |
| rating_score_avg    |  3.71851 | 0.465162 |      3.5 |  3.77116 |        4 |     1 |       5 |
| rating_score_std    | 0.377064 | 0.226758 | 0.227303 | 0.353553 | 0.485278 |     0 | 2.80014 |
| rating_score_median |  3.73267 | 0.468513 |      3.5 |     3.77 |        4 |     1 |       5 |
| nbr_interactions    |  30.0601 |  233.192 |        1 |        2 |        8 |     0 |   16509 |
| nbr_ratings         |  20.7816 |  169.233 |        0 |        1 |        5 |     0 |   12698 |
| nbr_reviews         |  9.27851 |  69.5388 |        0 |        1 |        2 |     0 |    3899 |
+---------------------+-------

## Computation of dropped values
Here we will compute how much data for each dataset has been dropped. This step is needed to ensure that we still have enough data to work with after the data cleaning process and to help spotting possible issues in the data cleaning process.
##### Users dataset

In [43]:
# Load the original datasets
df_users_original = pl.read_csv(f"{SRC_DATA_PATH}/users.csv")

# Do some computation
original_number_of_students = df_users_original.shape[0]
new_number_of_students = df_users.shape[0]

# Print the results
print(f"Number of students in the original dataset: {original_number_of_students}")
print(f"Number of students in the new dataset: {new_number_of_students}")
print(f"Percentage of students removed: {(original_number_of_students - new_number_of_students) / original_number_of_students * 100:.2f}%")

Number of students in the original dataset: 153704
Number of students in the new dataset: 153704
Percentage of students removed: 0.00%


##### Beers dataset

In [44]:
# Load the original datasets
df_beers_original = pl.read_csv(f"{SRC_DATA_PATH}/beers.csv")

# Do some computation
original_number_of_beers = df_beers_original.shape[0]
new_number_of_beers = df_beers.shape[0]

# Print the results
print(f"Number of beers in the original dataset: {original_number_of_beers}")
print(f"Number of beers in the new dataset: {new_number_of_beers}")
print(f"Percentage of beers removed: {(original_number_of_beers - new_number_of_beers) / original_number_of_beers * 100:.2f}%")

Number of beers in the original dataset: 280823
Number of beers in the new dataset: 273764
Percentage of beers removed: 2.51%


##### Breweries dataset

In [45]:
# Load the original datasets
df_breweries_original = pl.read_csv(f"{SRC_DATA_PATH}/breweries.csv")

# Do some computation
original_number_of_breweries = df_breweries_original.shape[0]
new_number_of_breweries = df_breweries.shape[0]

# Print the results
print(f"Number of beers in the original dataset: {original_number_of_breweries}")
print(f"Number of beers in the new dataset: {new_number_of_breweries}")
print(f"Percentage of beers removed: {(original_number_of_breweries - new_number_of_breweries) / original_number_of_breweries * 100:.2f}%")

Number of beers in the original dataset: 16758
Number of beers in the new dataset: 14028
Percentage of beers removed: 16.29%


##### Ratings dataset

In [46]:
# Load the original datasets
df_reviews_original = pl.read_parquet(f"{SRC_DATA_PATH}/ratings.pq")

# Do some computation
original_number_of_reviews = df_reviews_original.filter(pl.col("review")).shape[0]
original_number_of_ratings = df_reviews_original.filter(~pl.col("review")).shape[0]
new_number_of_reviews = df_ratings_no_text.filter(pl.col("review")).shape[0]
new_number_of_ratings = df_ratings_no_text.filter(~pl.col("review")).shape[0]

# Print the results
print(f"Number of reviews in the original dataset: {original_number_of_reviews}")
print(f"Number of reviews in the new dataset: {new_number_of_reviews}")
print(f"Percentage of reviews removed: {(original_number_of_reviews - new_number_of_reviews) / original_number_of_reviews * 100:.2f}%")
print()
print(f"Number of ratings in the original dataset: {original_number_of_ratings}")
print(f"Number of ratings in the new dataset: {new_number_of_ratings}")
print(f"Percentage of ratings removed: {(original_number_of_ratings - new_number_of_ratings) / original_number_of_ratings * 100:.2f}%")

Number of reviews in the original dataset: 2589586
Number of reviews in the new dataset: 2540122
Percentage of reviews removed: 1.91%

Number of ratings in the original dataset: 5803446
Number of ratings in the new dataset: 5689251
Percentage of ratings removed: 1.97%


##### Final discussion
In the data cleaning we see that while we don't lose lots of beers, users or ratings we lose lots of breweries (more than 16.29%!). While this seem at the first glance a lot we know that most of the dropped breweries are breweries without any beer associated with them. We can see, indeed, that the percentage of ratings lost is minimal (1.97 %) and we filter out all the ratings whose brewery id is not present in the brewery dataset after the cleaning process. As a consequence we can be sure that we still have enough data to work with.

## Conclusion
In this notebook we have cleaned the BeerAdvocate dataset and we now have data ready to be used for further analysis. <br>
Here we did focus on understanding the general structure of our data, clean them and produce processed files (in the format of parquet files) that can be easily loaded and used for further analysis.